## 3.6 层级索引

In [2]:
import pandas as pd
import numpy as np

### 3.6.1 多级索引Series

#### 1.笨办法

In [3]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

通过元组构成的多级索引，可以直接在Series上取值或者用切片查询数据：

In [5]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [6]:
pop[[i for i in pop.index if i[1] == 2010]] 

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

这么做虽然也能得到需要的结果，但是与 Pandas 令人爱不释手的切片语法相比，这种方法确实不够简洁（在处理较大的数据时也不够高效）。

#### 2.好办法：Pandas多级索引

In [10]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [13]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [26]:
pop[:,2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

#### 3.高维数据的多级索引 

In [27]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [28]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [29]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [32]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### 3.6.2 多级索引的创建方法

为 Series 或 DataFrame 创建多级索引最直接的办法就是将 index 参数设置为至少二维的索引数组，MultiIndex 的创建工作将在后台完成。

In [33]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.766858  0.567748
  2  0.296555  0.295958
b 1  0.407662  0.909687
  2  0.227298  0.521623

同理，如果你把将元组作为键的字典传递给 Pandas， Pandas 也会默认转换为 MultiIndex。

In [34]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

但是有时候显式地创建 MultiIndex 也是很有用的，下面来介绍一些创建方法。

#### 1.显式地创建多级索引

In [35]:
#你可以通过一个有不同等级的若干简单数组组成的列表来构建 MultiIndex：
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [36]:
#也可以通过包含多个索引值的元组构成的列表创建 MultiIndex：
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [37]:
#还可以用两个索引的笛卡尔积（Cartesian product）创建 MultiIndex：
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [38]:
#更可以直接提供 levels（包含每个等级的索引值列表的列表）和 
#labels（包含每个索引值 标签列表的列表）创建 MultiIndex：
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [41]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=pd.MultiIndex.from_product([['A', 'B'], [1, 2]]),
                  columns=['data1', 'data2'])
df

data1     data2
A 1  0.447675  0.142721
  2  0.780226  0.492485
B 1  0.821052  0.596052
  2  0.421492  0.603755

两种使用方法：在创建 Series 或 DataFrame 时，可以将这些对象**作为 index 参数**，或者**通过 reindex 方法 更新 Series 或 DataFrame 的索引**。

#### 2.多级索引的等级名称

两种起名方式：**在前面任何一个 MultiIndex 构造器中通过 names 参数设置等级名称**，也可以**在创建之后通过索引的 names 属性来修改名称**：


In [43]:
pop.index.names = ['state', 'year'] 
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [47]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=pd.MultiIndex.from_product([['A', 'B'], [1, 2]], names = ['name', 'number']),
                  columns=['data1', 'data2'])
df

data1     data2
name number                    
A    1       0.517904  0.291485
     2       0.890696  0.080490
B    1       0.816838  0.539958
     2       0.391248  0.375641

#### 3.多级列索引

In [52]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      27.0  35.4  32.0  35.5  56.0  38.1
     2      31.0  37.6  38.0  37.4  10.0  36.5
2014 1      42.0  37.2  36.0  36.6  47.0  36.2
     2      28.0  36.9  28.0  37.5  12.0  36.8

In [58]:
health_data['Guido'] 

type          HR  Temp
year visit            
2013 1      32.0  35.5
     2      38.0  37.4
2014 1      36.0  36.6
     2      28.0  37.5

### 3.6.3 多级索引的取值与切片

#### 1.Series多级索引

In [59]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [60]:
pop['California', 2000] 

33871648

In [61]:
pop['California'] 

year
2000    33871648
2010    37253956
dtype: int64

In [62]:
pop.loc['California':'New York'] 

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [63]:
pop[:, 2000] 

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [64]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [65]:
pop[['California', 'Texas']] 

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

#### 2.DataFrame多级索引

In [66]:
health_data 

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      27.0  35.4  32.0  35.5  56.0  38.1
     2      31.0  37.6  38.0  37.4  10.0  36.5
2014 1      42.0  37.2  36.0  36.6  47.0  36.2
     2      28.0  36.9  28.0  37.5  12.0  36.8

由于 DataFrame 的基本索引是列索引，因此 Series 中多级索引的用法到了 DataFrame 中就应用在列上了。

In [67]:
health_data['Guido', 'HR']

year  visit
2013  1        32.0
      2        38.0
2014  1        36.0
      2        28.0
Name: (Guido, HR), dtype: float64

In [68]:
health_data.iloc[:2, :2] 

subject      Bob      
type          HR  Temp
year visit            
2013 1      27.0  35.4
     2      31.0  37.6

In [70]:
health_data.loc[:, ('Bob', 'HR')] 

year  visit
2013  1        27.0
      2        31.0
2014  1        42.0
      2        28.0
Name: (Bob, HR), dtype: float64

In [71]:
idx = pd.IndexSlice         
health_data.loc[idx[:, 1], idx[:, 'HR']] 

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,27.0,32.0,56.0
2014,1,42.0,36.0,47.0


### 3.6.4 多级索引行列转换

#### 1.有序的索引和无序的索引 

In [72]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.744595
      2      0.927599
c     1      0.832294
      2      0.130867
b     1      0.672336
      2      0.846381
dtype: float64

**如果想对不按字典顺序排列的索引使用局部切片，那么错误就会出现。局部切片和许多其他相似的操作都要求 MultiIndex 的各级索引是有序的（即按照字典顺序由 A 至 Z）**

In [74]:
data = data.sort_index() 
data

char  int
a     1      0.744595
      2      0.927599
b     1      0.672336
      2      0.846381
c     1      0.832294
      2      0.130867
dtype: float64

In [75]:
data['a':'b'] 

char  int
a     1      0.744595
      2      0.927599
b     1      0.672336
      2      0.846381
dtype: float64

#### 2.索引stack与unstack 

In [76]:
pop.unstack(level=0) 

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [77]:
 pop.unstack(level=1) 

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [78]:
 pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### 3.索引的设置与重置 

In [79]:
pop_flat = pop.reset_index(name='population') 
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [80]:
pop_flat.set_index(['state', 'year']) 

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### 3.6.5 多级索引的数据累计方法

In [81]:
health_data 

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      27.0  35.4  32.0  35.5  56.0  38.1
     2      31.0  37.6  38.0  37.4  10.0  36.5
2014 1      42.0  37.2  36.0  36.6  47.0  36.2
     2      28.0  36.9  28.0  37.5  12.0  36.8

In [87]:
data_mean = health_data.mean(axis = 0, level='year') 
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     29.0  36.50  35.0  36.45  33.0  37.3
2014     35.0  37.05  32.0  37.05  29.5  36.5

In [88]:
data_mean.mean(axis = 1, level = 'type')

type,HR,Temp
year,,
2013,32.333333,36.750000
2014,32.166667,36.866667


## 3.7 合并数据集：Concat与Append操作

### 3.7.1 知识回顾：NumPy数组的合并 

In [89]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [90]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [91]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

### 3.7.2 通过pd.concat实现简易合并 

In [94]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [96]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1); print(df2); print(pd.concat([df1, df2])) 

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


In [99]:
df3 = make_df('AB', [0, 1])        
df4 = make_df('CD', [0, 1])       
print(df3)
print(df4)
print(pd.concat([df3, df4], axis='columns')) 

    A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


#### 1.索引重复

In [101]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
print(x); print(y); print(pd.concat([x, y])) 

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


##### (1)捕捉索引重复的错误。

##### (2)忽略索引。

In [102]:
print(x); print(y); print(pd.concat([x, y], ignore_index=True)) 

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


##### (3)增加多级索引。

In [103]:
print(x); print(y); print(pd.concat([x, y], keys=['x', 'y'])) 

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


#### 2.类似join的合并

In [112]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5)
print(df6)
print(pd.concat([df5, df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [113]:
print(pd.concat([df5, df6], join='inner')) 

    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


In [114]:
print(pd.concat([df5, df6], join_axes=[df5.columns])) 

     A   B   C
1   A1  B1  C1
2   A2  B2  C2
3  NaN  B3  C3
4  NaN  B4  C4


#### 3.append()方法

In [115]:
print(df1.append(df2)) 

    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


## 3.8 合并数据集：合并与连接

### 3.8.1 关系代数

### 3.8.2 数据连接的类型 

#### 1.一对一连接

In [116]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1); print(df2) 

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [118]:
df3 = pd.merge(df1, df2) 
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### 2. 多对一连接 

In [119]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'], 
                    'supervisor': ['Carly', 'Guido', 'Steve']})        
print(df3); print(df4); print(pd.merge(df3, df4)) 

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


#### 3. 多对多连接

In [120]:
 df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',                                      'Engineering', 'Engineering', 'HR', 'HR'], 
                     'skills': ['math', 'spreadsheets', 'coding', 'linux',  
                     'spreadsheets', 'organization']}) 
print(df1); print(df5); print(pd.merge(df1, df5)) 
 

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


### 3.8.3 设置数据合并的键 

#### 1. 参数on的用法

In [121]:
print(df1); print(df2); print(pd.merge(df1, df2, on='employee')) 

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


#### 2.left_on与right_on参数

In [122]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]}) 
print(df1); print(df3); print(pd.merge(df1, df3, left_on="employee", right_on="name")) 

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


In [123]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1) 

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### 3.left_index与right_index参数

In [124]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')        
print(df1a); print(df2a) 

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


In [125]:
print(pd.merge(df1a, df2a, left_index=True, right_index=True)) 

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


In [127]:
print(df1a.join(df2a)) 

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


In [128]:
print(df1a); print(df3); print(pd.merge(df1a, df3, left_index=True, right_on='name')) 

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000


### 3.8.4 设置数据连接的集合操作规则

In [129]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])         
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])         
print(df6); print(df7); print(pd.merge(df6, df7)) 

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


In [131]:
pd.merge(df6, df7, how='inner') 

,name,food,drink
0,Mary,bread,wine


In [132]:
print(pd.merge(df6, df7, how='outer')) 

     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


In [133]:
print(df6); print(df7); print(pd.merge(df6, df7, how='left')) 

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


### 3.8.5 重复列名：suffixes参数 

In [134]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})         
print(df8); print(df9); print(pd.merge(df8, df9, on="name")) 

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [135]:
print(pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])) 

   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


### 3.8.6 案例：美国各州的统计数据 

In [12]:
import pandas as pd
pop = pd.read_csv('PythonDataScienceHandbook-master/notebooks/data/state-population.csv')
areas = pd.read_csv('PythonDataScienceHandbook-master/notebooks/data/state-areas.csv')
abbrevs = pd.read_csv('PythonDataScienceHandbook-master/notebooks/data/state-abbrevs.csv')
print(pop.head()); print(areas.head()); print(abbrevs.head()) 

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


首先用一个多对一合并获取人口（pop）DataFrame 中各州名称缩写对应的全称。我们需要将 pop 的 state/region 列与 abbrevs 的 abbreviation 列进行合并，还需要通过 how='outer' 确保数据没有丢失。


In [14]:
merged = pd.merge(pop, abbrevs, how = 'outer', left_on = 'state/region', right_on = 'abbreviation')
merged = merged.drop('abbreviation', 1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [15]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [16]:
merged[merged['population'].isnull()].head() 

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [18]:
merged.loc[merged['state'].isnull(), 'state/region'].unique() 

array(['PR', 'USA'], dtype=object)

In [21]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'         
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'         
merged.isnull().any() 

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [23]:
final = pd.merge(merged, areas, on = 'state', how = 'left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [24]:
final.isnull().any() 

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [25]:
final['state'][final['area (sq. mi)'].isnull()].unique() 

array(['United States'], dtype=object)

In [26]:
final.dropna(inplace=True)         
final.head() 

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [27]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [48]:
data2010.set_index('state/region', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)'] 

In [51]:
density.sort_values(ascending=False, inplace=True)         
density.head() 

state/region
DC    8898.897059
PR    1058.665149
NJ    1009.253268
RI     681.339159
CT     645.600649
dtype: float64

In [52]:
density.tail() 

state/region
SD    10.583512
ND     9.537565
MT     6.736171
WY     5.768079
AK     1.087509
dtype: float64

>当人们用现实世界的数据解决问题时，合并这类脏乱的数据是十分常见的任务。希望这个案例可以帮你把前面介绍过的工具串起来，从而在数据中找到想要的答案！